In [ ]:
# 1. Katie Corcoran

In [1]:
# 2.
import os
import pandas as pd

path = '/Users/katiecorcoran/Dropbox/5330corgis'
files = os.listdir(path)
for file in files:
    print(file)

102121-7380.csv
pdx623.csv
102121-5390.csv
portlandeventjun22.csv
newreg-062822.csv
102123-3428.csv
102124-9377.csv
Icon
newcorgis-20220619.csv
newcorgis-20220618.csv
signupsjune272022.csv
102124-3988.csv
102119-8469.csv
tacoma-062322.csv
102122-4015.csv
102120-8243.csv


In [38]:
# 3.
import psycopg2

conn = psycopg2.connect(
database='corgirace',
user='kcorcoran',
password='mydata5600db',
host='database-1.cjykoq2oos56.us-east-2.rds.amazonaws.com',
port= '5432')
cursor = conn.cursor()

issueslist = []
for file in files:
    # Skip if not a csv
    if file[-3:] != 'csv':
        print(f'Skipped file {file}')
        continue
    
    # Skip if already ingested
    cursor.execute(f'''
        SELECT
            COUNT(*)
        FROM ingest
        WHERE filename = '{file}';
    ''')
    y = cursor.fetchone()
    if y[0] > 0:
        print(f'{file} has already been processed')
        continue
    
    infile = open(f'{path}/{file}', 'r')
    data = infile.readlines()
    for line in data[1:]:
        issueslist = []
        row = line.strip().split(',')
        if (len(row) > 5):
            issueslist.append('Too Many Columns')
            continue
        if (len(row) < 5):
            issueslist.append('Too Few Columns')
            continue
        # Name
        name = row[0]
        if (len(name) < 1):
            issueslist.append('Name Too Short')
        elif (len(name) > 100):
            issueslist.append('Name Too Long')   
        # Breed
        breed = row[1]
        if len(breed) < 1 or (breed[0].lower() != 'p' and breed[0].lower() != 'c'):
            issueslist.append('Breed Issue')
        # Gender
        valid = ['f', 'm', 'sf', 'nm']
        if row[2].lower() not in valid:
            issueslist.append('Gender Col Issue')
        # Weight
        try:
            weight = float(row[3])
            if (weight < 15):
                issueslist.append('Weight Too Low')
            elif (weight > 35):
                issueslist.append('Weight Too High')
        except:
            issueslist.append('Weight Non-Numeric')
        # Age
        try:
            age = float(row[4])
            if (age < 1):
                issueslist.append('Age Too Low')
            elif (age > 10):
                issueslist.append('Age Too High')
        except:
            issueslist.append('Age Non-Numeric') 

        print(f'File Name: {file}')
        print(f'Name: {name}')
        print(issueslist)
        
            
        

102121-7380.csv has already been processed
pdx623.csv has already been processed
102121-5390.csv has already been processed
portlandeventjun22.csv has already been processed
newreg-062822.csv has already been processed
102123-3428.csv has already been processed
102124-9377.csv has already been processed
Skipped file Icon
newcorgis-20220619.csv has already been processed
newcorgis-20220618.csv has already been processed
signupsjune272022.csv has already been processed
102124-3988.csv has already been processed
102119-8469.csv has already been processed
tacoma-062322.csv has already been processed
102122-4015.csv has already been processed
102120-8243.csv has already been processed


In [14]:
# 4.
cursor.execute('''CREATE TABLE IF NOT EXISTS corgi_exception (
    exceptid SERIAL PRIMARY KEY,
    except_record TEXT,
    origin_file VARCHAR(100) NOT NULL,
    issues TEXT,
    except_timestamp TIMESTAMP,
    fixed_timestamp TIMESTAMP);''')
conn.commit()

In [39]:
# 5.
issueslist = []
for file in files:
    # Skip if not a csv
    if file[-3:] != 'csv':
        print(f'Skipped file {file}')
        continue
    
    # Skip if already ingested
    cursor.execute(f'''
        SELECT
            COUNT(*)
        FROM ingest
        WHERE filename = '{file}';
    ''')
    y = cursor.fetchone()
    if y[0] > 0:
        print(f'{file} has already been processed')
        continue
    
    infile = open(f'{path}/{file}', 'r')
    data = infile.readlines()
    for line in data[1:]:
        issueslist = []
        row = line.strip().split(',')
        if (len(row) > 5):
            issueslist.append('Too Many Columns')
        elif (len(row) < 5):
            issueslist.append('Too Few Columns')
        else:
            # Name
            name = row[0]
            if (len(name) < 1):
                issueslist.append('Name Too Short')
            elif (len(name) > 100):
                issueslist.append('Name Too Long')   
            # Breed
            breed = row[1]
            if len(breed) > 1:
                if (breed[0].lower() == 'p'):
                    breed = 'Pem'
                elif (breed[0].lower() == 'c'):
                    breed = 'Cardi'
                else:
                    issueslist.append('Breed Issue')
            else:
                issueslist.append('Breed Issue')
            # Gender
            gender = row[2]
            valid = ['f', 'm', 'sf', 'nm']
            if row[2].lower() not in valid:
                issueslist.append('Gender Col Issue')
            # Weight
            try:
                weight = float(row[3])
                if (weight < 15):
                    issueslist.append('Weight Too Low')
                elif (weight > 35):
                    issueslist.append('Weight Too High')
            except:
                issueslist.append('Weight Non-Numeric')
            # Age
            try:
                age = float(row[4])
                if (age < 1):
                    issueslist.append('Age Too Low')
                elif (age > 10):
                    issueslist.append('Age Too High')
            except:
                issueslist.append('Age Non-Numeric')
                
        if (len(issueslist) == 0):
            cursor.execute(f'''
                INSERT INTO corgi (corgname, breed, gender, weight, age, fromfile)
                VALUES (
                '{name}',
                '{breed}',
                '{gender}',
                {round(weight)},
                {round(age * 2) / 2},
                '{file}');
            ''')
            conn.commit()
            print(f"Record created for {name}.")
        else:
            cursor.execute(f'''
                INSERT INTO corgi_exception (except_record, origin_file, issues, except_timestamp)
                VALUES ('{'|'.join(row).replace("'", '')}',
                '{file}',
                '{'|'.join(issueslist)}',
                CURRENT_TIMESTAMP);
            ''')
            conn.commit()
            print(f"Record for {row[0]} had these issues: {', '.join(issueslist)}")
            
    cursor.execute(f'''
            INSERT INTO ingest (filename, whendone)
            VALUES ('{file}', CURRENT_TIMESTAMP);
        ''')
    conn.commit()
    print(f"Ingestion complete for {file}.")

Record for Speedbump had these issues: Too Many Columns
Record for Bandit had these issues: Too Many Columns
Record for Bailey Barkbark had these issues: Too Many Columns
Record for Trucky McDog had these issues: Too Many Columns
Record for Oakley Ann had these issues: Too Many Columns
Record for Maverick had these issues: Too Many Columns
Record for Rancher had these issues: Too Many Columns
Record for Bowser had these issues: Too Many Columns
Record for  had these issues: Too Many Columns
Record for  had these issues: Too Many Columns
Ingestion complete for 102121-7380.csv.
pdx623.csv has already been processed
Record created for Governess Moonbeam.
Record created for Drumstick.
Record created for Punchy Judy.
Record created for Hywel the Woofer.
Record for Jerry Brown had these issues: Gender Col Issue
Ingestion complete for 102121-5390.csv.
portlandeventjun22.csv has already been processed
newreg-062822.csv has already been processed
Record created for Brunchkiller.
Record created 

In [40]:
# 6.
df = pd.read_sql('''
    SELECT
        exceptid,
        except_record,
        origin_file
    FROM corgi_exception
    ORDER BY except_record
    LIMIT 10
''', conn, index_col='exceptid')
print(df)

                            except_record      origin_file
exceptid                                                  
297                Amelia the Woof|||27|3  102120-8243.csv
274              Baby Congrats|PWC|M|36|4  102123-3428.csv
265       Bailey Barkbark|Cardi|SF|26|2||  102121-7380.csv
264                   Bandit|Pem|M|31|4||  102121-7380.csv
270                  Bowser|Pem|NM|27|4||  102121-7380.csv
275                      |Cardi|SF|26|4.5  102123-3428.csv
295                   Charles|Pem||36|2.5  102120-8243.csv
288               Crossbow Jo|Pem|R|5|2.7  102124-3988.csv
286        Double-Dragon|Cardi|NM|32|3.5|  102124-9377.csv
292         Double-Plus|Pem|SF|Heavy!|4.5  102122-4015.csv


/var/folders/44/s9hsvm1943z9xb042mqxxtgc0000gn/T/ipykernel_18904/3996143579.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''


In [41]:
# 7.
df = pd.read_sql('''
    SELECT
        corgid,
        corgname,
        weight,
        age,
        fromfile
    FROM corgi
    JOIN ingest ON ingest.filename = corgi.fromfile
    WHERE ingest.whendone::date = CURRENT_DATE
    ORDER BY corgname
    LIMIT 10
''', conn, index_col='corgid')
print(df)

                corgname  weight  age         fromfile
corgid                                                
483            Baby Bear      29  3.0  102122-4015.csv
469         Brunchkiller      27  3.5  102123-3428.csv
472            Campstove      22  2.0  102123-3428.csv
492         Corgi Elliot      24  3.0  102120-8243.csv
484             Corgihor      28  3.5  102122-4015.csv
473      Delbert the Cog      31  4.0  102124-3988.csv
486         Demi Locorgo      25  3.5  102122-4015.csv
466            Drumstick      23  2.5  102121-5390.csv
490             Empanada      26  1.5  102120-8243.csv
478     Finnleberry Huck      26  3.0  102119-8469.csv


/var/folders/44/s9hsvm1943z9xb042mqxxtgc0000gn/T/ipykernel_18904/2337558358.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''
